In [16]:
import urllib.request as request
from bs4 import BeautifulSoup
import pandas
import time
from selenium import webdriver
import glob
import os
import re
import ntpath
from random import randint
from PIL import Image
import numpy as np
from captcha_solver import CaptchaSolver
solver = CaptchaSolver("rucaptcha", api_key='')
import urllib
import numpy
from selenium.common.exceptions import NoSuchElementException, NoSuchWindowException, WebDriverException 
import shutil
from selenium.common.exceptions import WebDriverException
import threading
import random

In [45]:
class SearchDOI:
    def __init__(this, folder_to_download=r"C:\Users\Maryia_Ivanina\cornell_project\pdfs_to_download",
                file_with_urls="urls_to_download.xlsx"):
        this.folder_to_download = folder_to_download
        os.makedirs(folder_to_download, exist_ok=True)
        prefs = {"download.default_directory": folder_to_download,
        "download.prompt_for_download": False,
        "plugins.always_open_pdf_externally": True}
        
        chromeOptions = webdriver.ChromeOptions()
        prefs = {"download.default_directory" : folder_to_download}
        chromeOptions.add_experimental_option("prefs",prefs)
        chromedriver = r"C:\chromedriver.exe"
        
        ##save driver for later use 
        this.driver = webdriver.Chrome(executable_path=chromedriver, chrome_options=chromeOptions)
        this.driver.delete_all_cookies()
        
        ## save pandas for later use 
        this.csv = pandas.read_excel(file_with_urls)
        this.csv =  this.csv[['title', 'id','url']]
        
        ## place holders
        this.title = str
        this.cov_id = int
        
    def start_downloading(this):
        ## new column
        this.csv['automation_file_name'] = 'not found'

        this.num_files_downloaded = 0
    
        ##start download
        this.csv.apply(this.download, axis=1)
        
        this.csv.to_csv('searchDone.csv')  
        
    def captcha_exists(this): 
        try:
            if this.driver.find_element_by_id('captcha'):
                pass
            else:
                raise ValueError('Item not found on page') 
            return True 
        except:
            return False 
            
    def slove_captcha(this):
        try:
            src = this.driver.find_element_by_id("captcha").get_attribute("src") ## captcha
            ## download captcha
            request.urlretrieve(src, "captcha.jpg")

            ## slove the captcha
            solver = CaptchaSolver('rucaptcha', api_key='03a58a9552a6972819384712f3ede9e1')
            raw_data = open('captcha.jpg', 'rb').read()
            code = solver.solve_captcha(raw_data)

            ##put code 
            this.driver.find_element_by_name("answer").send_keys(code)
            this.driver.find_element_by_xpath('//input[@value="Продолжить"]').click()
            
            if(this.driver.find_elements_by_css_selector('#buttons > ul > li:nth-child(2) > a')): ## download button 
                time.sleep(3)
                this.driver.find_element_by_xpath('//*[@id="buttons"]/ul/li[2]/a').click() 
                time.sleep(6)
                this.check_for_download()
                
            return True 
        except:
            return False
        
        
    def rename_downloaded_file(this, cov_id):
        list_of_files = glob.glob(r"%s\\*" % this.folder_to_download) # * means all if need specific format then *.pdf
        latest_file = max(list_of_files, key=os.path.getctime)

        ## old filename
        file_name = ntpath.basename(latest_file)
        
        this.title = re.sub(r'[\\/*?:"<>|]',"",this.title)
        this.title = this.title[:200]

        ## new file name
        #updated_file_name = latest_file.replace(file_name, 
        #                                        ("{}_{}.pdf".format(this.title, str(cov_id).replace('#', ''))))
        updated_file_name = latest_file.replace(file_name, "{}.pdf".format(cov_id))
        shutil.move(latest_file, updated_file_name)
            
        
    def num_files_dir(this):
        dir_files = os.listdir(this.folder_to_download)
        return len(dir_files)
    
    def pdf_exists(this):
        try:
            this.driver.find_element_by_xpath("//a[contains(@href,'.pdf')]")
            return True 
        except NoSuchElementException:
            return False 
    
    def check_for_download(this):
        if this.num_files_dir() > this.num_files_downloaded: ## there has been a file dowloaded
            this.num_files_downloaded +=1
            this.rename_downloaded_file(this.cov_id)

            list_of_files = glob.glob(r"%s\\*" % this.folder_to_download) # * means all if need specific format then *.pdf
            latest_file = max(list_of_files, key=os.path.getmtime)
            file_name = ntpath.basename(latest_file)
            
            this.csv.loc[this.csv['id'] == this.cov_id, "automation_file_name"] = "found"
            print("Downloaded ", this.cov_id)
            this.csv.to_csv('Search_done_progress.csv') 
            
    # watch for change in folder 
    def download_wait(this):
        seconds = 0
        dl_wait = True
        while dl_wait and seconds < 120:
            time.sleep(1)
            dl_wait = False
            for fname in os.listdir(r"%s\\" % this.folder_to_download):
                if fname.endswith('.crdownload'):
                    dl_wait = True
            seconds += 1
        return True
    
    def download(this, row, backoff=3):
        while backoff > 0:
            try:
                this.download_row(row)
                backoff = 0
            except:
                time.sleep(120)
                backoff -= 1

    def download_row(this, row):
        print(row.name)
        this.cov_id = row['id']
        this.title = row['title']
        if type(row['url']) == str:
            this.driver.get('https://sci-hub.se/');
            search_box = this.driver.find_element_by_xpath('//*[@id="input"]/form/input[2]')
            search_box.send_keys(row['url'])
            search_box.submit()
            
            if(this.driver.find_elements_by_css_selector('#buttons > button')): ## download button 
                #print(this.driver.find_elements_by_css_selector('#buttons > ul > li:nth-child(2) > a'))
                #print(this.driver.find_element_by_xpath('//*[@id="buttons"]/ul/li[2]/a'))
                this.driver.find_element_by_xpath('//*[@id="buttons"]/button').click()

                if this.download_wait():
                    this.check_for_download()   
                if this.captcha_exists():
                    print('chapta')
                    if this.captcha_exists():
                        if this.slove_captcha():
                            if this.download_wait():
                                this.check_for_download()
            elif(this.pdf_exists()): ## article pdf link
                try:
                    time.sleep(3)
                    this.driver.find_element_by_xpath("//a[contains(@href,'.pdf')]").click()
                except:
                    pass

                if this.download_wait():
                    this.check_for_download()
            else:  ## auto downloaded
                if this.download_wait():
                    this.check_for_download()


In [46]:
SearchDOI().start_downloading()

C:\Users\Maryia_Ivanina\Anaconda3\envs\env-pdf-download\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: use options instead of chrome_options
  app.launch_new_instance()


0
Downloaded  1
1
Downloaded  2
2
Downloaded  3
3
Downloaded  4
4
Downloaded  5
5
Downloaded  6
6
Downloaded  7
7
Downloaded  8
8
Downloaded  9
9
Downloaded  10
10
11
11
Downloaded  12
12
Downloaded  13
13
Downloaded  14
14
Downloaded  15
15
Downloaded  16
16
Downloaded  17
17
Downloaded  18
18
Downloaded  19
19
Downloaded  20


In [134]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

pages = []
with open(r'C:\Users\Maryia_Ivanina\cornell_project\pdfs_to_download\3.pdf', 'rb') as in_file:
    parser = PDFParser(in_file)
    doc = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    for page in PDFPage.create_pages(doc):
        output_string = StringIO()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        
        interpreter.process_page(page)
        pages.append(output_string.getvalue())

In [137]:
from pdfminer.pdfdocument import PDFDocument, PDFNoOutlines
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTPage, LTChar, LTAnno, LAParams, LTTextBox, LTTextLine

class PDFPageDetailedAggregator(PDFPageAggregator):
    def __init__(self, rsrcmgr, pageno=1, laparams=None):
        PDFPageAggregator.__init__(self, rsrcmgr, pageno=pageno, laparams=laparams)
        self.rows = []
        self.page_number = 0
    def receive_layout(self, ltpage):        
        def render(item, page_number):
            if isinstance(item, LTPage) or isinstance(item, LTTextBox):
                for child in item:
                    render(child, page_number)
            elif isinstance(item, LTTextLine):
                child_str = ''
                for child in item:
                    if isinstance(child, (LTChar, LTAnno)):
                        child_str += child.get_text()
                child_str = ' '.join(child_str.split()).strip()
                if child_str:
                    row = (page_number, item.bbox[0], item.bbox[1], item.bbox[2], item.bbox[3], child_str) # bbox == (x1, y1, x2, y2)
                    self.rows.append(row)
                for child in item:
                    render(child, page_number)
            return
        render(ltpage, self.page_number)
        self.page_number += 1
        self.rows = sorted(self.rows, key = lambda x: (x[0], -x[2]))
        self.result = ltpage

In [315]:
from pprint import pprint
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from time import time

t_start = time()
pages = []
with open(r'C:\Users\Maryia_Ivanina\cornell_project\Auto-search-and-download-articles-by-DOI-using-SciHub-master\Auto-search-and-download-articles-by-DOI-using-SciHub-master\pdfs_downloaded\1\4327.pdf', 'rb') as fp:
    parser = PDFParser(fp)
    doc = PDFDocument(parser)

    rsrcmgr = PDFResourceManager()
    laparams = LAParams()
    device = PDFPageDetailedAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    for pid, page in enumerate(PDFPage.create_pages(doc)):
        try:
            interpreter.process_page(page)
            # receive the LTPage object for this page
            layout = device.get_result()
            text = []
            
            for lt_obj in sorted(layout, key=lambda x:(x.bbox[0] >= 300, -x.bbox[1])):
                try:
                    #print(lt_obj.bbox)
                    #print(lt_obj.get_text())
                    text.append(lt_obj.get_text())
                except:
                    pass
            pages.append("\n".join(text))
        except Exception as err:
            print(err)
print(time() - t_start)

1.3508961200714111


In [303]:
t_start = time()
# funding source
text_to_use = []
for page in range(len(pages)-1, -1, -1):
    if "acknowledgment" in pages[page].lower() or "acknowledgement" in pages[page].lower():
        all_text = "\n\n".join(pages[page:])
        start = False
        for line in all_text.split("\n"):
            line_to_check = line
            if line.isupper():
                line_to_check.replace(" ", "")
            if "acknowledgment" in line.lower() or "acknowledgement" in line.lower():
                start = True
            if "acknowledgment" in line_to_check.lower() or "acknowledgement" in line_to_check.lower():
                start = True
            found_new_section = False
            for key_word in ["reference", "appendix", "declaration", "note"]:
                if line.lower().startswith(key_word):
                    found_new_section = True
                    break
            if found_new_section:
                if start:
                    start = False
                    break
            if start:
                text_to_use.append(line)
        break
print("\n".join(text_to_use))
print(time() - t_start)


0.001999378204345703


In [326]:
import shutil
cnt = 0
cnt_all = 0
for i in range(500):
    if not os.path.exists("pdfs_downloaded\%s" % i):
        continue
    if i >= 20 and i <= 99:
        continue
    if len(os.listdir("pdfs_downloaded\%s" % i)) >= 0 and len(os.listdir("pdfs_downloaded\%s" % i)) <= 30:
        print(i, len(os.listdir("pdfs_downloaded\%s" % i)))
        cnt += 1
        cnt_all += len(os.listdir("pdfs_downloaded\%s" % i))
        #shutil.copy("urls_to_find_funders/%s.xlsx" % i, "urls_to_find_funders_left/%s.xlsx" % i)
        #shutil.copytree("pdfs_downloaded\%s" % i, "pdfs\%s" % i)
cnt, cnt_all

3 30
7 22
17 16
161 30
184 29
189 13
199 29
212 15
286 17
308 10
317 24
356 19
360 15
368 22
374 26
419 28
420 21
421 27
422 18
457 16
473 28


(21, 455)

In [334]:
cnt_all = 0
cnt_with_ack = 0
for file in os.listdir("folder_with_ack_full/"):
    df = pandas.read_excel(os.path.join("folder_with_ack_full", file)).fillna("")
    for i in range(len(df)):
        if df["acknowledgement_part"].values[i].strip():
            cnt_with_ack += 1
        cnt_all += 1
cnt_with_ack, cnt_all

(33356, 39117)

In [335]:
cnt_with_ack / cnt_all

0.8527238796431219